In [69]:
import pandas as pd
import os

# 입력 및 출력 디렉토리 설정
input_directory = "large_data"
output_file = "cmm_data.csv"

# 빈 리스트 생성
deviation_rows = []
quality_status = []
file_names = []

# 입력 디렉토리 내의 모든 파일에 대해 반복
for filename in os.listdir(input_directory):
    if filename.endswith(".csv"):
        # CSV 파일 읽기
        file_path = os.path.join(input_directory, filename)
        df = pd.read_csv(file_path, delimiter=',', encoding='cp949', index_col=False)  # index_col=False 설정

        # '항목' 열의 값이 "SMmf"인 행 제외
        df = df[df['항목'] != "SMmf"]

        # '도형_항목' 열 생성
        df['도형_항목'] = df['도형'] + ',' + df['항목']

        # 결측된 편차 값을 '-' 대신 'NaN'으로 바꾸기
        df['편차'] = df['편차'].replace('-', pd.NA)

        # 결측된 편차 값에 대해 평균 값 계산
        avg_deviation = pd.to_numeric(df['편차'], errors='coerce').mean(skipna=True)  # 'NaN' 값은 자동으로 무시됨

        # 결측된 편차 값을 평균 값으로 대체
        df['편차'] = df['편차'].fillna(avg_deviation)

        # 편차 값 추출
        deviations = df['편차'].tolist()

        # 파일명 저장
        file_names.append(os.path.splitext(filename)[0])  # 파일명에서 확장자 제외하여 저장

        # 품질상태 열 추가
        if df.iloc[1, 16] == 'OK':  # 두 번째 열이 'OK'인 경우
            quality_status.append(1)
        else:
            quality_status.append(0)

        # 리스트에 추가
        deviation_rows.append(deviations)

# 편차를 행으로, 도형_항목을 열로 배치하여 데이터프레임 생성
combined_data = pd.DataFrame(deviation_rows, columns=df['도형_항목'])  # '도형_항목' 열을 사용하여 DataFrame 생성

# 파일명을 첫 번째 열로 추가
combined_data.insert(0, '파일명', file_names)

# 품질상태 열 추가
combined_data['품질상태'] = quality_status

# 결과를 CSV 파일로 저장
combined_data.to_csv(output_file, encoding='cp949', index=False)  # 인덱스 저장 안 함


<ipython-input-69-029dde76f3ba>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['도형_항목'] = df['도형'] + ',' + df['항목']
<ipython-input-69-029dde76f3ba>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['편차'] = df['편차'].replace('-', pd.NA)
<ipython-input-69-029dde76f3ba>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/